In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Example 1: Chipotle Locations

In [ ]:
import json
df=pd.read_json('/kaggle/input/chipotle-locations/us-states.json')
df.head()

Reads json into a dataframe, but based on how the json is structured, only two columns are created and much information is embedded deep in the dataframe.

In [ ]:
df['type'].value_counts()

This column is useless--> let's look at the features column that contains the embedded data.

In [ ]:
df['features'].head()

This is a list of dictionaries (dictionaries are like lists, but contain specified keys/headers). This is the data we actually want.

Let's look at the first list item and see how the data is stored.

In [ ]:
df['features'][0]

In [ ]:
df['features'][0].keys()

Let's say the information we want is the id (state abbreviation), state name, geometry type, and the number of coordinate pairs provided in that given state.

In [ ]:
len(df['features'][0]['geometry']['coordinates'][0])

In [ ]:
frame=pd.DataFrame()
for i in range(len(df['features'])):
    ID=df['features'][i]['id']
    state=df['features'][i]['properties']['name']
    shape=len(df['features'][i]['geometry']['coordinates'])
    geo=df['features'][i]['geometry']['type']
    shape=len(df['features'][i]['geometry']['coordinates'])
    if shape>1:
        num=0
        for j in range(shape):
            num+=len(df['features'][i]['geometry']['coordinates'][j][0])
    else:
        num=len(df['features'][i]['geometry']['coordinates'][0])
    frame=frame.append({'id':ID,'state':state,'geometry':geo,'shape':shape,'coordinate pairs':num},ignore_index=True)  

In [ ]:
frame

# Example 2: German Recipes

In [ ]:
ger=pd.read_json('/kaggle/input/german-recipes-dataset/recipes.json')
ger.head()

This came out much cleaner than the previous file- the JSON file is set up in such a way that the keys directly lead to the information (there are not multiple levels of organization in which we have to dig for the data we want).

In [ ]:
ger['Ingredients'][0]

Only minor thing is that the ingredients are stored as a list. Let's add to this dataset by adding a new column for the number of ingredients required.

In [ ]:
ger['num']=[len(ger['Ingredients'][i]) for i in range(len(ger))]
ger.head()

Idea: Let's find a Sauerkraut recipe with the least amount of ingredients.

In [ ]:
sk=ger[ger['Name'].str.contains("Sauerkraut")]
sk.head()

In [ ]:
len(sk)

66 sauerkraut recipes in this dataset

In [ ]:
Min=sk[sk['num']==sk['num'].min()]
Min.head()

In [ ]:
len(Min)

Minumum number of ingredients is 12, 29 sauerkraut recipes have 12 ingredients. 

Now which one of these has the least amount of instructions?

In [ ]:
#compute number of letters as length of instructions
len(Min.reset_index()['Instructions'][0])

In [ ]:
pd.options.mode.chained_assignment = None
Min['length']=[len(Min['Instructions'][index]) for index in Min.reset_index()['index']]
Min

Actually it looks like these are all repeats. Going to have to revert back and get rid of all the duplicates. This will significantly reduce the number of entries in the dataframe.

In [ ]:
fix=ger.drop_duplicates(subset=['Instructions'])
len(fix)

In [ ]:
sk2=fix[fix['Name'].str.contains("Sauerkraut")]
sk2

Only two unique sauerkraut recipes here. Let's try to analytically determine which has more instructions.

In [ ]:
sk2['letters']=[len(sk2['Instructions'][index]) for index in sk2.reset_index()['index']]
sk2['words']=[len(sk2['Instructions'][index].split()) for index in sk2.reset_index()['index']]
sk2

Looks like the top one is the better recipe given our priorities.